In [73]:
import librosa
import soundfile
import os
import glob
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [74]:
def extract_features(file_name, mfcc, chroma, mel):
    """
    Extracts features (mfcc, chroma, mel) from a sound file.
    :param file_name: str, path to the sound file
    :param mfcc: bool, whether to extract mfcc features
    :param chroma: bool, whether to extract chroma features
    :param mel: bool, whether to extract mel features
    :return: np.array, extracted features
    """
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

In [75]:
# Mapping of emotions in the RAVDESS dataset to labels
emotions = {
  '01': 'neutral',
  '02': 'calm',
  '03': 'happy',
  '04': 'sad',
  '05': 'angry',
  '06': 'fearful',
  '07': 'disgust',
  '08': 'surprised'
}

In [76]:
# Emotions to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

def load_data(test_size=0.2):
    """
    Loads the data and extracts features for each sound file.
    :param test_size: float, fraction of data to use for testing
    :return: tuple of np.arrays, train and test data and labels
    """
    x,y = [],[]
    for file in glob.glob(".\\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_features(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=18)

In [77]:
# Split the dataset into training and testing sets
x_train,x_test,y_train,y_test = load_data(test_size=0.20)

In [78]:
# Print the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [79]:
# Print the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [80]:
# Initialize the Multi Layer Perceptron Classifier
model = MLPClassifier(alpha=0.01,
                      batch_size=256,
                      epsilon=1e-08,
                      hidden_layer_sizes=(800,),
                      learning_rate='adaptive',
                      max_iter=1000)

In [81]:
# Train the model on the training data
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(800,),
              learning_rate='adaptive', max_iter=1000)

In [82]:
# Make predictions on the test data
y_pred = model.predict(x_test)

In [83]:
# Calculate the accuracy of the model on the test data
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

In [84]:
# Print the accuracy of the model on the test data
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.13%
